<a href="https://colab.research.google.com/github/eric88525/Tbrain_NLP_game/blob/master/train_BIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 洗錢新聞分類器

In [ ]:
!nvidia-smi

Fri Jul 31 18:30:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
!pip install transformers
!pip install opencc
!pip install pyprind
!pip install zhon
#!pip install -U ckiptagger[tfgpu,gdown]

In [ ]:
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
from time import strftime,gmtime
from opencc import OpenCC
import pyprind
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
from zhon.hanzi import non_stops,stops
import numpy as np
import random
print(torch.cuda.is_available())
!nvidia-smi

True
Fri Jul 31 18:30:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [ ]:
cd ./drive/My Drive/Colab Notebooks/TBrain/nosep

/content/drive/My Drive/Colab Notebooks/TBrain/nosep


In [ ]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)

In [ ]:
class mydata_nosep():
    def __init__(self,train_path,test_path):

        self.cc = OpenCC('t2s')

        bin = lambda p: 1 if len(p) !=0 else 0

        train_data = pd.read_csv(train_path)
        test_data = pd.read_csv(test_path)

        # CONTENT / bin
        train_x = [ self.clean_string(i) for i in train_data['full_content'].values.tolist()]
        train_y = [ bin(self.name_list(i)) for i in train_data['name'].values.tolist()]

        # CONTENT / bin
        test_x = [self.clean_string(i) for i in test_data['full_content'].values.tolist()]
        test_y = [ bin(self.name_list(i)) for i in test_data['name'].values.tolist() ]

        self.train = [] # CONTENT / NAME / (CKIP-NAME)
        self.test = []  # CONTENT / NAME

        qs = 0 
        for i in range(len(train_x)): 
          self.train.append([train_x[i],train_y[i]])
        
        for i in range(len(test_x)):
          self.test.append([test_x[i],test_y[i]])


        print(f'Train: {len(self.train)} Test: {len(self.test)}')

    def name_list(self,name_list_str):
        ls = ast.literal_eval(name_list_str)
        res = []
        cc = OpenCC('t2s')
        for n in ls:
          res.append(cc.convert(n))
        return set(res)        
    def clean_string(self,content):
      content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
      content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
      content = re.sub(r"[%s]+" %stops, "。",content)
      content = self.cc.convert(content)
      return content

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TrainDataset(Dataset):
    def __init__(self,data,model_type):
      self.data = data
      self.tokenizer = BertTokenizer.from_pretrained(model_type)
        
    def __getitem__(self,idx):
      paragraph ,label = self.data[idx][0],self.data[idx][1]
      token_tensor = self.tokenizer.encode_plus(paragraph,paragraph,max_length=512,truncation=True,pad_to_max_length=True)
      label_tensor = torch.Tensor([label])
      # token / segment / mask / label
      return  torch.tensor(token_tensor['input_ids']), torch.tensor( token_tensor['token_type_ids']) , torch.tensor( token_tensor['attention_mask'] ),label_tensor
    def __len__(self):
      return len(self.data)


In [ ]:
"""
class bertwwmBIN(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmBIN,self).__init__()
        self.bert_model = BertModel.from_pretrained(model_name)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,2)
        )
    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary
"""

'\nclass bertwwmBIN(nn.Module):\n    def __init__(self,model_name,config):\n        super(bertwwmBIN,self).__init__()\n        self.bert_model = BertModel.from_pretrained(model_name)\n        self.bi_decoder = nn.Sequential(\n            nn.Linear(config.hidden_size,config.hidden_size)\n            ,nn.Dropout(0.1)\n            ,nn.ReLU()\n            ,nn.Linear(config.hidden_size,2)\n        )\n    def start(self):\n      nn.init.xavier_uniform_(self.bi_decoder[0].weight)\n      nn.init.constant_(self.bi_decoder[0].bias, 0)\n      nn.init.xavier_uniform_(self.bi_decoder[3].weight)\n      nn.init.constant_(self.bi_decoder[3].bias, 0)\n\n    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):\n        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]\n        binary = self.bi_decoder(cls)\n        return binary\n'

In [ ]:
class bertwwmBIN(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmBIN,self).__init__()
        self.bert_model = BertModel.from_pretrained(model_name)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,1)
            ,nn.Sigmoid()
        )
    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary

In [ ]:
class bertwwmQA(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmQA,self).__init__()
        #self.bert_model = BertModel.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
        
        self.bert_model = BertModel.from_pretrained(model_name,config = config)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,2)
        )
        #self.start()

    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
       # print(f'receive input_ids {input_ids}')

        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary

In [ ]:
"""
def test(model,data,device,model_name):
    print('Testing...')
    test_num = len(data.test)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model.eval()
    print(f'Testing data: {test_num}')
    all,acc = 0,0
    with torch.no_grad():
      for content,label in data.test:
        all += 1
        if len(content)>512:
          content = content[:512] 
        token_tensor = tokenizer.encode_plus(str(content),max_length=512,truncation=True,pad_to_max_length=True)
        token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(device)
        segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(device)
        mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(device)
        pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
        print(pred)
        if pred.item() == 1 and label == 1:
          acc = acc +1
        elif pred.item() == 0 and label == 0:
          acc = acc +1
    print(f'ALL: {all} ACC: {acc}')       
    return all,acc
"""

"\ndef test(model,data,device,model_name):\n    print('Testing...')\n    test_num = len(data.test)\n    tokenizer = BertTokenizer.from_pretrained(model_name)\n    model.eval()\n    print(f'Testing data: {test_num}')\n    all,acc = 0,0\n    with torch.no_grad():\n      for content,label in data.test:\n        all += 1\n        if len(content)>512:\n          content = content[:512] \n        token_tensor = tokenizer.encode_plus(str(content),max_length=512,truncation=True,pad_to_max_length=True)\n        token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(device)\n        segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(device)\n        mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(device)\n        pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)\n        print(pred)\n        if pred.item() == 1 and label == 1:\n          acc = acc +1\n        elif pred.item() == 0 and label == 0:\n          

In [ ]:
def test(model,data,device,model_name):
    print('Testing...')
    test_num = len(data.test)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model.eval()
    print(f'Testing data: {test_num}')
    all,acc = 0,0
    with torch.no_grad():
      for content,label in data.test:
        all += 1
        if len(content)>512:
          content = content[:512] 
        token_tensor = tokenizer.encode_plus(str(content),max_length=512,truncation=True,pad_to_max_length=True)
        token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(device)
        segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(device)
        mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(device)
        pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment)
        print(f'P {pred.item():.2f} L {label}')
        if pred.item() >0.4 and label == 1:
          acc = acc +1
        elif pred.item() < 0.4 and label == 0:
          acc = acc +1
    print(f'ALL: {all} ACC: {acc}')       
    return all,acc

In [ ]:
"""
def train(trainLoader,w_d,lr_rate,device,model_type,epoches):
  config = BertConfig.from_pretrained(model_type, output_hidden_states=True)
  model = bertwwmBIN(model_type,config).to(device)
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)
  #criterion = nn.BCELoss()
  criterion = nn.CrossEntropyLoss()
  model.train()
  for e in range(epoches):
    loss,i = 0,0
    for data in trainLoader:
      i+=1
      token , segment , mask , label = [ item.to(device) for item in data]
      pred = torch.softmax(model(input_ids=token,attention_mask=mask,token_type_ids=segment),dim=-1)
      
      label = label.reshape(label.shape[0])
     # print(pred)
     # print(label)

     # print(pred.shape,label.shape)
      batch_loss = criterion(pred,label.long())
      loss += batch_loss
     # print(batch_loss)
      batch_loss.backward()
     # print(f'Epoch {e} Batch {i} Loss is {batch_loss}')
      optimizer.step()
      optimizer.zero_grad()
     # print(f'Loss {batch_loss}')
    print(f'Epoches: {e} Loss {loss} AVG: {loss/i}')
  best_model = copy.deepcopy(model.state_dict())
  return best_model
"""

"\ndef train(trainLoader,w_d,lr_rate,device,model_type,epoches):\n  config = BertConfig.from_pretrained(model_type, output_hidden_states=True)\n  model = bertwwmBIN(model_type,config).to(device)\n  parameters = filter(lambda p: p.requires_grad, model.parameters())\n  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)\n  #criterion = nn.BCELoss()\n  criterion = nn.CrossEntropyLoss()\n  model.train()\n  for e in range(epoches):\n    loss,i = 0,0\n    for data in trainLoader:\n      i+=1\n      token , segment , mask , label = [ item.to(device) for item in data]\n      pred = torch.softmax(model(input_ids=token,attention_mask=mask,token_type_ids=segment),dim=-1)\n      \n      label = label.reshape(label.shape[0])\n     # print(pred)\n     # print(label)\n\n     # print(pred.shape,label.shape)\n      batch_loss = criterion(pred,label.long())\n      loss += batch_loss\n     # print(batch_loss)\n      batch_loss.backward()\n     # print(f'Epoch {e} Batch {i} Loss is {batch_loss}')\

In [ ]:
def train(trainLoader,w_d,lr_rate,device,model_type,epoches):
  config = BertConfig.from_pretrained(model_type, output_hidden_states=True)
  model = bertwwmBIN(model_type,config).to(device)
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = AdamW(parameters, lr=lr_rate, weight_decay=w_d)
  criterion = nn.BCELoss()
  #criterion = nn.CrossEntropyLoss()
  model.train()
  for e in range(epoches):
    loss,i = 0,0
    for data in trainLoader:
      i+=1
      token , segment , mask , label = [ item.to(device) for item in data]
      pred = model(input_ids=token,attention_mask=mask,token_type_ids=segment)
      #print(f'Pred {pred} {pred.shape}')
     # print(f'Label {label} {label.shape}')
      batch_loss = criterion(pred,label)
      loss += batch_loss
     # print(batch_loss)
      batch_loss.backward()
     # print(f'Epoch {e} Batch {i} Loss is {batch_loss}')
      optimizer.step()
      optimizer.zero_grad()
     # print(f'Loss {batch_loss}')
    print(f'Epoches: {e} Loss {loss} AVG: {loss/i}')
  best_model = copy.deepcopy(model.state_dict())
  return best_model

In [ ]:
mydata = mydata_nosep('./data0/tbrain_train.csv','./data0/tbrain_test.csv') # 491 / 487
#mydata = mydata_nosep('./data1/tbrain_train.csv','./data1/tbrain_test.csv') # 491 / 491
#mydata = mydata_nosep('./data2/tbrain_train.csv','./data2/tbrain_test.csv') # 491 / 491
#mydata = mydata_nosep('./data3/tbrain_train.csv','./data3/tbrain_test.csv') # 491 / 491

Train: 4426 Test: 491


In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
mode = 'testBIN'
model_type = 'hfl/chinese-roberta-wwm-ext'
#model_type = "hfl/chinese-bert-wwm"
torch.cuda.empty_cache()  

In [ ]:
if mode == 'trainBIN':
  train_ds = TrainDataset(mydata.train,model_type)
  trainLoader = DataLoader(train_ds, batch_size=8)
  config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
  print('Train BIN')
  best_model = train(trainLoader,0.001,1e-5,device,model_type,5)
  if not os.path.exists('saved_models'):
    os.makedirs('saved_models')    
  modeltime = strftime('%H_%M_%S', gmtime()) 
  modelname = 'bertWWM_BIN_'+ modeltime
  torch.save(best_model, f'saved_models/{modelname}.pt')
  print(f'Train end, model name is {modelname}.pt')

elif mode == 'testBIN':
  print('testbin')
  #model_type = 'hfl/chinese-roberta-wwm-ext'
  #modelname = 'bertWWM_BIN_09_05_44.pt'
  modelname = 'bertWWM_BIN_18_21_15.pt'
  config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
  test_model = bertwwmBIN(model_type,config).to(device)
  test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
  test(test_model,mydata,device,model_type)

testbin
Testing...
Testing data: 491
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 1.00 L 1
P 0.00 L 0
P 1.00 L 1
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.95 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 1.00 L 1
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.34 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 1.00 L 1
P 1.00 L 1
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00 L 0
P 0.00

In [ ]:
#RoBERTa-wwm-ext-large	hfl/chinese-roberta-wwm-ext-large
#RoBERTa-wwm-ext	hfl/chinese-roberta-wwm-ext
#BERT-wwm-ext	hfl/chinese-bert-wwm-ext
#BERT-wwm	hfl/chinese-bert-wwm
#RBT3	hfl/rbt3
#RBTL3	hfl/rbtl3